# 🎮 AI Trivia Game Show - Week 2 Exercise

An interactive trivia game that demonstrates Week 2 concepts:

- **Gradio Chat UI** - Interactive game interface
- **Streaming Responses** - Real-time message generation
- **System Prompts** - Game show host personality
- **Model Switching** - Compare different GPT models
- **Score Tracking** - Maintain state across conversation

The AI generates trivia questions and evaluates your answers!

In [1]:
# Install required packages (run once)
# !pip install gradio openai

In [2]:
import gradio as gr
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
# Game state
game_state = {
    "score": 0,
    "questions_answered": 0,
    "correct_answers": 0
}

# Available models
MODELS = {
    "GPT-4o": "gpt-4o",
    "GPT-4o Mini": "gpt-4o-mini",
    "GPT-3.5 Turbo": "gpt-3.5-turbo"
}

In [ ]:
def chat(message, history, model_name, category, difficulty):
    """Main chat function with streaming responses"""
    
    # System prompt that defines the AI's personality and behavior
    system_prompt = f"""You are an enthusiastic trivia game show host! 

RULES:
1. When the user wants to play, generate ONE trivia question based on:
   - Category: {category if category != "Any Category" else "any topic you choose"}
   - Difficulty: {difficulty if difficulty != "Any Difficulty" else "medium"}

2. Present the question in an exciting, engaging way
3. After they answer, tell them if they're RIGHT or WRONG
4. Award points: Easy=10pts, Medium=20pts, Hard=30pts
5. Keep it fun and energetic like a real game show host!

Current Score: {game_state['score']} points
Questions Answered: {game_state['questions_answered']}
Correct Answers: {game_state['correct_answers']}

IMPORTANT: Only ask ONE question at a time. Wait for their answer before asking another.
"""
    
    # Build messages array with system prompt
    messages = [{"role": "system", "content": system_prompt}]
    
    # Add conversation history
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        if assistant_msg:
            messages.append({"role": "assistant", "content": assistant_msg})
    
    # Add current user message
    messages.append({"role": "user", "content": message})
    
    # Stream the AI response
    response_text = ""
    
    stream = client.chat.completions.create(
        model=MODELS[model_name],
        messages=messages,
        stream=True,
        temperature=0.8  # Higher temperature for more creative/entertaining responses
    )
    
    # Yield response chunks as they arrive
    for chunk in stream:
        if chunk.choices[0].delta.content:
            response_text += chunk.choices[0].delta.content
            yield response_text
    
    # Update game state based on AI's response
    response_lower = response_text.lower()
    
    # Check if answer was correct
    if "correct" in response_lower or "right" in response_lower or "yes" in response_lower:
        if game_state["questions_answered"] < len(history) + 1:
            # Determine points based on difficulty
            points = 10 if difficulty == "Easy" else (30 if difficulty == "Hard" else 20)
            game_state["score"] += points
            game_state["questions_answered"] += 1
            game_state["correct_answers"] += 1
    
    # Check if answer was wrong
    elif "wrong" in response_lower or "incorrect" in response_lower or "not quite" in response_lower:
        if game_state["questions_answered"] < len(history) + 1:
            game_state["questions_answered"] += 1

In [5]:
def reset_game():
    """Reset the game state"""
    game_state["score"] = 0
    game_state["questions_answered"] = 0
    game_state["correct_answers"] = 0
    return f"Game reset! Score: 0 | Questions: 0"

def get_score():
    """Get current score display"""
    return f"Score: {game_state['score']} pts | Correct: {game_state['correct_answers']}/{game_state['questions_answered']}"

In [ ]:
# Build Gradio interface
with gr.Blocks(title="AI Trivia Game", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎮 AI Trivia Game Show")
    gr.Markdown("Your AI host will quiz you with trivia questions. Answer correctly to score points!")
    
    with gr.Row():
        # Left column - Chat interface
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(height=400, label="Game Show Host")
            
            with gr.Row():
                msg = gr.Textbox(
                    label="Your Message",
                    placeholder="Say 'Let's play!' to start...",
                    scale=4
                )
                submit = gr.Button("Send", scale=1, variant="primary")
            
            score_display = gr.Textbox(
                label="Score", 
                value=get_score(), 
                interactive=False
            )
        
        # Right column - Settings
        with gr.Column(scale=1):
            gr.Markdown("### ⚙️ Settings")
            
            model_selector = gr.Dropdown(
                choices=list(MODELS.keys()),
                value="GPT-4o Mini",
                label="AI Model"
            )
            
            category_selector = gr.Dropdown(
                choices=[
                    "Any Category", "Science", "History", "Geography", 
                    "Sports", "Entertainment", "Music", "Literature"
                ],
                value="Any Category",
                label="Category"
            )
            
            difficulty_selector = gr.Dropdown(
                choices=["Any Difficulty", "Easy", "Medium", "Hard"],
                value="Medium",
                label="Difficulty"
            )
            
            reset_btn = gr.Button("🔄 Reset Game", variant="stop")
            
            gr.Markdown("### 📊 Scoring")
            gr.Markdown("""
            - Easy: 10 pts
            - Medium: 20 pts  
            - Hard: 30 pts
            """)
    
    # Event handlers
    def respond(message, chat_history, model, category, difficulty):
        """Handle user message and update UI"""
        bot_response = ""
        for response in chat(message, chat_history, model, category, difficulty):
            bot_response = response
        
        chat_history.append((message, bot_response))
        return "", chat_history, get_score()
    
    def reset_all():
        """Reset game and clear chat"""
        reset_game()
        return [], get_score()
    
    # Connect events to handlers
    msg.submit(
        respond, 
        [msg, chatbot, model_selector, category_selector, difficulty_selector],
        [msg, chatbot, score_display]
    )
    
    submit.click(
        respond, 
        [msg, chatbot, model_selector, category_selector, difficulty_selector],
        [msg, chatbot, score_display]
    )
    
    reset_btn.click(reset_all, None, [chatbot, score_display])

# Launch the app
demo.launch()

## How to Play

1. Run all cells above
2. Say "Let's play!" or "Give me a trivia question"
3. Answer the question
4. Get points for correct answers!
5. Try different categories, difficulties, and models

## Week 2 Concepts Demonstrated

- **Gradio UI**: Interactive chat interface
- **Streaming**: Real-time response generation  
- **System Prompt**: Game show host personality
- **Model Switching**: Compare GPT-4o, GPT-4o Mini, GPT-3.5
- **State Management**: Score tracking across conversation